<a href="https://colab.research.google.com/github/Erike-Simon/CESAR-AED/blob/main/ProcDados_spark_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=b737a85eaefce80a6609db826a94f3250398ceab51f904dd598a6af943e208c0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## Dados

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Path:**  
data/ad_action.csv

## Atividade 1

Qual é o anúncio mais popular, ou seja, com mais eventos independente do action?

**Resposta:**  
adId_06

## Atividade 2

Quantos clicks gerou a campanha mais popular?

**Resposta:**  
63983 clicks

## Atividade 3

Algum usuário só visualizou? Quantas actions foram enviadas por usuário que só visualizou?

**Resposta:**  
Sim, o usuário d99871cb-98b7-4ac5-97a5-b9a26c0f897b enviou apenas 1 action

## Atividade 4

Dos 10411 usuários, quantos usuários clicam mais que visualizam?

**Resposta:**  
10078

## Atividade 5

Calcule a quantidade de clicks por dia da semana e apresente o resultado em ordem decrescente de quantidade de clicks.

PS: Nesse exemplo, 0 é segunda-feira e 6 é domingo. Você pode usar outro padrão, contanto que os dias da semana estejam correspondentes a resposta abaixo

**Resposta**  
27918, 4  
25424, 5  
25028, 0  
25027, 3  
25020, 6  
24973, 1  
24915, 2

## Atividade 6

Algum anúncio teve um crescimento de clicks ao longo das 3 campanhas?

**Resposta:**  
Sim, o anúncio adId_07

In [ ]:
# IMPORTS, CRIAÇÃO DO CLUSTER E CARREGAMENTO DOS DADOS AQUI
import numpy as np
import os
import pandas as pd
import pyspark.sql.functions as F

from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS'] = '\
    --driver-memory 4G \
    --executor-memory 4G \
    pyspark-shell'

In [ ]:
# Criando um cluster local com 1 executor e a quantidade de threads igual a quantidade de cores de CPU disponíveis

spark = SparkSession.builder\
    .master("local[*]")\
    .getOrCreate()
spark

In [ ]:
# Comando para desativar os recursos do spark
# spark.stop()

In [ ]:
AD_ACTION_CSV_PATH = 'drive/MyDrive/data/ad_action.csv'

In [ ]:
df = spark.read.csv(AD_ACTION_CSV_PATH, header=False, inferSchema=True)\
    .toDF('timestamp', 'user_id', 'action', 'adId', 'campaignId')
df.show(3)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 3 rows



In [ ]:
df.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- action: string (nullable = true)
 |-- adId: string (nullable = true)
 |-- campaignId: string (nullable = true)



## Resposta Atividade 1

Qual é o anúncio mais popular, ou seja, com mais eventos independente do action?

**Resposta:**  
adId_06

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

In [ ]:
df.groupby('adId')\
    .count()\
    .orderBy(F.desc('count'))\
    .take(1)[0]['adId']

'adId_06'

## Resposta Atividade 2

Quantos clicks gerou a campanha mais popular?

**Resposta:**  
63983 clicks

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

In [ ]:
most_popular_campaign = df.groupBy('campaignId')\
    .count()\
    .orderBy(F.desc('count'))\
    .take(1)[0]['campaignId']

df.where((F.col('campaignId') == most_popular_campaign) & (F.col('action') == "click"))\
    .count()

63983

## Resposta Atividade 3

Algum usuário só visualizou? Quantas actions foram enviadas por usuário que só visualizou?

**Resposta:**  
Sim, o usuário d99871cb-98b7-4ac5-97a5-b9a26c0f897b enviou apenas 1 action

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

In [ ]:
df.show(3)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 3 rows



In [ ]:
df.groupBy('user_id', 'action')\
    .count()\
    .groupBy('user_id')\
    .pivot('action')\
    .sum('count')\
    .where(F.col('click').isNull())\
    .toPandas()

,user_id,click,view
0,d99871cb-98b7-4ac5-97a5-b9a26c0f897b,NaN,1


## Resposta Atividade 4

Dos 10411 usuários, quantos usuários clicam mais que visualizam?

**Resposta:**  
10078

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

In [ ]:
df.show(3)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 3 rows



In [ ]:
df.groupBy('user_id', 'action')\
    .count()\
    .groupBy('user_id')\
    .pivot('action')\
    .sum('count')\
    .na.fill(0)\
    .where(F.col('click') > F.col('view'))\
    .count()

10078

## Resposta Atividade 5

Calcule a quantidade de clicks por dia da semana e apresente o resultado em ordem decrescente de quantidade de clicks.

PS: Nesse exemplo, 0 é segunda-feira e 6 é domingo. Você pode usar outro padrão, contanto que os dias da semana estejam correspondentes a resposta abaixo

**Resposta**  
27918, 4  
25424, 5  
25028, 0  
25027, 3  
25020, 6  
24973, 1  
24915, 2

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

In [ ]:
df.show(3)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 3 rows



In [ ]:
result = df.withColumn('day_of_week', F.dayofweek('timestamp'))\
    .where(F.col('action') == "click")\
    .groupBy('day_of_week')\
    .count()\
    .orderBy(F.desc('count'))\
    .collect()

print("\n")
for row in result:
    print(f"{row['count']}: {row['day_of_week']}")



27918: 6
25424: 7
25028: 2
25027: 5
25020: 1
24973: 3
24915: 4


## Resposta Atividade 6

Algum anúncio teve um crescimento de clicks ao longo das 3 campanhas?

**Resposta:**  
Sim, o anúncio adId_07

In [ ]:
# ESCREVA SEU CÓDIGO AQUI

In [ ]:
df.show(3)

+-------------------+--------------------+------+-------+-------------+
|          timestamp|             user_id|action|   adId|   campaignId|
+-------------------+--------------------+------+-------+-------------+
|2016-09-21 22:11:00|7c74953c-66cc-48b...| click|adId_09|campaignId_01|
|2016-06-25 18:29:00|676a083e-2f8e-4ff...|  view|adId_09|campaignId_02|
|2016-02-14 19:03:00|77158997-0dfa-48b...| click|adId_02|campaignId_02|
+-------------------+--------------------+------+-------+-------------+
only showing top 3 rows



In [ ]:
df.select('adId').distinct().count()

10

In [ ]:
df.select('campaignId').distinct().count()

3

In [ ]:
df.select('campaignId', 'adId').distinct().count()

29

In [ ]:
ads_df = df.where(df['action'].like("click"))\
    .groupBy('campaignId', 'adId')\
    .count()\
    .withColumnRenamed('count', 'clicks_count')\
    .groupBy('adId')\
    .pivot('campaignId')\
    .sum('clicks_count')\
    .na.fill(0)\
    .orderBy('adId')
ads_df.show()

+-------+-------------+-------------+-------------+
|   adId|campaignId_01|campaignId_02|campaignId_03|
+-------+-------------+-------------+-------------+
|adId_01|         5156|         6452|         5361|
|adId_02|         5073|         7423|         6509|
|adId_03|         5719|         7046|         5197|
|adId_04|         5025|         7023|         6348|
|adId_05|         5518|         6815|         5818|
|adId_06|         5533|         7923|         7074|
|adId_07|         5287|         6909|         7081|
|adId_08|         5451|         6900|         5689|
|adId_09|         5529|         7492|         5857|
|adId_10|         5084|            0|         6013|
+-------+-------------+-------------+-------------+



In [ ]:
result_df = ads_df.where(
    (ads_df['campaignId_01'] < ads_df['campaignId_02']) &\
    (ads_df['campaignId_02'] < ads_df['campaignId_03'])
)
result_df.show()

+-------+-------------+-------------+-------------+
|   adId|campaignId_01|campaignId_02|campaignId_03|
+-------+-------------+-------------+-------------+
|adId_07|         5287|         6909|         7081|
+-------+-------------+-------------+-------------+



In [ ]:
print(f"Sim, o anúncio {result_df.select('adId').collect()[0]['adId']}")

Sim, o anúncio adId_07
